### Initialization

In [ ]:
import os
import sys
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from models.mog_model import *
from utils.distributions import *

#plt.style.use('/Users/saforem2/.config/matplotlib/stylelib/dark_jupyter.mplstyle')
#plt.rcParams['figure.facecolor'] = '#474747'

%matplotlib notebook
%autoreload 2

In [ ]:
plt.style.use('default')

In [ ]:
import matplotlib as mpl
mpl.rcdefaults()

In [ ]:
def fix_legends(axes):
    if isinstance(axes, (np.ndarray, list)):
        legends = [ax.get_legend() for ax in axes]
        for leg in legends:
            leg.texts[0].set_color('w')
    else:
        legend = axes.get_legend()
        for idx in range(len(legend.texts)):
            legend.texts[idx].set_color('w')
    return axes 

In [ ]:
def add_vline(axes, x, **kwargs):
    if isinstance(axes, (np.ndarray, list)):
        for ax in axes:
            ax.axvline(x, **kwargs)
                       #, color='C3', ls=':', lw=2.)
    else:
        axes.axvline(x, **kwargs)
    return axes

In [ ]:
def fix_ticks(axes):
    if isinstance(axes, (np.ndarray, list)):
        for idx in range(len(axes)):
            axes[idx].tick_params(which='both', 
                                  color='#474747', 
                                  labelcolor='k')
    else:
        axes.tick_params(which='both', color='#474747', labelcolor='k')

In [ ]:
params = {'x_dim': 2,
          'num_distributions': 2,
          'eps': 0.1,
          'scale': 0.1,
          'num_samples': 100,
          'means': None,
          'sigma': 0.05,
          'small_pi': 2E-16,
          'lr_init': 1e-2,
          'temp_init': 10,
          'annealing_steps': 200,
          'annealing_factor': 0.98,
          'num_training_steps': 20000,
          'tunneling_rate_steps': 1000,
          'save_steps': 1000,
          'lr_decay_steps': 2500,
          'lr_decay_rate': 0.96,
          'logging_steps': 100,
          'arrangement': 'single_axis'}

### Gaussians separated along $x$-axis

In [ ]:
x_dim = 2
sigma = 0.05
centers = 1
means = np.zeros((x_dim, x_dim), dtype=np.float32)
means[::2, 0] = centers
means[1::2, 0] = -centers
arrangement='single_axis'
#for i in range(x_dim):
#    means[i::x_dim, i] = centers
cov_mtx = sigma * np.eye(x_dim).astype(np.float32)
covs = np.array([cov_mtx] * x_dim).astype(np.float32)
dist_arr = distribution_arr(x_dim, 2)
gmm_dist = GMM(means, covs, dist_arr)
params['means'] = gmm_dist.mus
params['x_dim'] = x_dim
params['sigma'] = sigma
params['arrangement'] = arrangement
#gmm_mus = np.array(gmm_dist.mus)
#gmm_diffs = gmm_mus[1:] - gmm_mus[:-1, :]
#gmm_distances = [np.sqrt(np.dot(d, d.T)) for d in gmm_diffs]

In [ ]:
covs

In [ ]:
%matplotlib notebook

In [ ]:
gmm_samples = gmm_dist.get_samples(500)
fig, ax = plt.subplots()
ax.plot(gmm_samples[:,0], gmm_samples[:,1],
        marker='o', ls='', alpha=0.75, label='Target distribution')
ax.legend(loc='best')
#_ = fix_legends(ax)
#_ = fix_ticks(ax)
fig.savefig('/Users/saforem2/ANL/l2hmc/new_mog_logs/target_distribution.pdf', 
            dpi=400, bbox_inches='tight')
plt.show()

### Ring of Gaussians

In [ ]:
x_dim = 2
num_distributions = 6
sigma = 0.01

#MEANS = np.zeros((X_DIM, X_DIM), dtype=np.float32)
#CENTERS = np.sqrt(2)  # center of Gaussian
#for i in range(NUM_DISTRIBUTIONS):
#    MEANS[i::NUM_DISTRIBUTIONS, i] = CENTERS

covs, distribution = gen_ring(r=1., var=sigma, nb_mixtures=num_distributions)
mus = np.array(distribution.mus)
diffs = mus[1:] - mus[:-1, :]
distances = [np.sqrt(np.dot(d, d.T)) for d in diffs]

### Lattice of Gaussians

In [ ]:
x_dim = 2
num_distributions = 16
var = 0.01
L = int(np.sqrt(num_distributions))
means = np.array([(i, j) for i in range(L) for j in range(L)])
_sigmas = np.array([var * np.eye(x_dim) for _ in range(num_distributions)])
pis = [1. / num_distributions] * num_distributions 
pis[0] += 1 - sum(pis)

In [ ]:
distribution = GMM(means, _sigmas, pis)

### Plot samples from distribution

In [ ]:
%matplotlib notebook
samples = distribution.get_samples(500)
fig, ax = plt.subplots()
ax.plot(samples[:,0], samples[:,1], marker='o', ls='', alpha=0.75)
plt.show()
#plt.savefig('../log_mog_tf/run_326/figures/target_distribution.pdf', dpi=400, bbox_inches='tight')

### GMM separated along diagonal

In [ ]:
x_dim = 2
sigma = 0.02
centers = 1
means = np.zeros((x_dim, x_dim), dtype=np.float32)
for i in range(x_dim):
    means[i::x_dim, i] = centers
cov_mtx = sigma * np.eye(x_dim).astype(np.float32)
covs = np.array([cov_mtx] * x_dim).astype(np.float32)
dist_arr = distribution_arr(x_dim, 2)
gmm_dist = GMM(means, covs, dist_arr)

gmm_mus = np.array(gmm_dist.mus)
gmm_diffs = gmm_mus[1:] - gmm_mus[:-1, :]
gmm_distances = [np.sqrt(np.dot(d, d.T)) for d in gmm_diffs]
gmm_distances

In [ ]:
gmm_samples = gmm_dist.get_samples(500)
fig, ax = plt.subplots()
ax.plot(gmm_samples[:,0], gmm_samples[:,1], marker='o', ls='', alpha=0.75)
plt.show()

### Create GMM model for training with L2HMC

In [ ]:
#x_dim = 2
#num_distributions = 6
#sigma = 0.005

#MEANS = np.zeros((X_DIM, X_DIM), dtype=np.float32)
#CENTERS = np.sqrt(2)  # center of Gaussian
#for i in range(NUM_DISTRIBUTIONS):
#    MEANS[i::NUM_DISTRIBUTIONS, i] = CENTERS

#covs, distribution = gen_ring(r=1.0, var=sigma, nb_mixtures=num_distributions)
#means = distribution.mus

params = {                          # default parameter values
    'x_dim': x_dim,
    'num_distributions': num_distributions,
    'means': means,
    'sigma': 0.05,
    'small_pi': 2E-16,
    'scale': 0.1,
    'num_samples': 200,
    'lr_init': 1e-3,
    'lr_decay_steps': 1000,
    'lr_decay_rate': 0.96, 'eps': 0.5,
    'temp_init': 20,
    'annealing_steps': 200,
    'annealing_rate': 0.98,
    #'train_trajectory_length': 15,
    #'test_trajectory_length': 2000,
    'num_training_steps': 30000,
    'tunneling_rate_steps': 1000,
    'save_steps': 1000,
    'logging_steps': 100
}

## Build / Train model

In [ ]:
tf.reset_default_graph()

In [ ]:
#%pdb
config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#kwargs = {'radius': 1.0, 'sigma': 0.01, 'num_distributions': 6}
#params = {}
model = GaussianMixtureModel(params, 
                             config=config,
                             log_dir='../../new_mog_logs/run_4',
                             covs=covs,
                             distribution=gmm_dist)
                             #**kwargs)
                             #log_dir='../log_mog_tf/run_22_diag_271/')
                             #log_dir='../log_mog_tf/run64/')
model.means = gmm_dist.mus

In [ ]:
model._restore_model()

## Train model

In [ ]:
model.train(40000)

In [ ]:
%debug

In [ ]:
target_samples = model.distribution.get_samples(500)

In [ ]:
trajectories, losses, px = model.generate_trajectories(temp=1., 
                                                       num_samples=10, 
                                                       num_steps=250)

In [ ]:
trajectories

In [ ]:
trajectories.shape

In [ ]:
#idxs = [rand_traj(trajectories) for _ in range(5)]
colors = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']
idxs = np.arange(7)
nums = np.arange(len(idxs))
labels = ['trajectory ' + str(i) for i in nums]
for num, idx in enumerate(idxs):
    fig, ax = plt.subplots()
    _ = ax.plot(target_samples[:,0], target_samples[:,1], 
                marker='o', ls='', alpha=0.75, color='slategrey')
    #for i in range(3):
    #for num, idx in enumerate(idxs):
    _ = ax.plot(trajectories[:, idx, 0], trajectories[:, idx, 1], 
                marker='.', ls='-', alpha=0.5, color=colors[num])#, label=labels[num], color=colors[num])
    #_ = ax.plot(trajectories[:, idxs[0], 0], trajectories[:, idxs[0], 1], marker='.', ls='-', alpha=0.5)#, label=labels[num], color=colors[num])
    #ax.legend(loc='lower left')
    fig.tight_layout()
    plt.show()
    #plt.savefig(f'../log_mog_tf/run_327/figures/trajectory_{num}.pdf', dpi=400, bbox_inches='tight')

## Testing dynamics

In [ ]:
step = 1
t = model.dynamics._format_time(step, tile=tf.shape(model.x)[0])
grad1 = model.dynamics.grad_energy(model.x, aux=None)
S1 = model.dynamics.VNet([model.x, grad1, t, None])

In [ ]:
sv1 = 0.5 * model.dynamics.eps * S1[0]
tv1 = S1[1]
fv1 = model.dynamics.eps * S1[2]

In [ ]:
from utils.dynamics import safe_exp
prod_sv1 = tf.multiply(model.z, safe_exp(sv1, name='sv1F'))
prod_fv1 = tf.multiply(safe_exp(fv1, name='fv1F'), grad1)
v_h = prod_sv1 + 0.5 * model.dynamics.eps * (-prod_fv1 + tv1)

In [ ]:
model.sess.run(v_h, feed_dict={model.x: samples, model.dynamics.temperature: 1.}).shape

In [ ]:
model.sess.run(tf.global_variables_initializer())

In [ ]:
model.sess.run(vnet, feed_dict={model.x: samples, model.dynamics.temperature: 1.})

In [ ]:
model._restore_model()

In [ ]:
model.train(10)

In [ ]:
trajectories, loss_arr, accept_arr = model.generate_trajectories()

In [ ]:
#trajectories[:, 0, :]
trajectories[:20,0, :]

In [ ]:
_samples = model.distribution.get_samples(600)
trajectories = []
for step in range(100):
    _samples = model.distribution.get_samples(200)
    trajectories.append(np.copy(_samples))
    feed_dict = {model.x: _samples,
                 model.dynamics.temperature: 1.}
    _samples, px = model.sess.run([
        #model.loss,
        model.output[0],
        model.px,
    ], feed_dict=feed_dict)

#  _, loss_, model.samples, px_, lr_, = model.sess.run([
#_, loss_, _samples, px_, lr_, = model.sess.run([
#    model.train_op,
#    model.loss,
#    model.output[0],
#    model.px,
#    model.learning_rate
#], feed_dict=feed_dict)

In [ ]:
_samples = model.distribution.get_samples(600)

In [ ]:
trajectories = np.array(trajectories)
np.array(trajectories).shape

In [ ]:
trajectories[:20, 0, :]

In [ ]:
from utils.notebook_utils import get_hmc_samples

In [ ]:
samples = np.random.randn(200, 2)
hmc_samples = get_hmc_samples(2, 0.1, gmm_dist.get_energy_function(),
                              model.sess, trajectory_length=10., steps=500, 
                              samples=samples)

In [ ]:
hmc_samples.shape

In [ ]:
trajectories.shape

In [ ]:
#for i in range(10):
fig, ax = plt.subplots()
ax.plot(_samples[:,0], _samples[:,1], ls='', marker='.', alpha=0.75)
ax.plot(trajectories[:50, 8, 0], trajectories[:50, 8, 1], 
        ls='-', marker='', alpha=0.6, label='L2HMC Chain')
ax.plot(hmc_samples[:50, 0, 0], trajectories[:50, 0, 1], 
        ls='-', marker='', alpha=0.6, color='C3', label='HMC Chain')
ax.legend(loc='best')
plt.savefig('../../new_mog_logs/l2hmc_vs_hmc_trajectories.pdf', 
            dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
model.global_step.eval(model.sess)

In [ ]:
model.arrangement = 'axes'

In [ ]:
model._generate_plots(model.global_step.eval(model.sess))

In [ ]:

_samples1 = model.sess.run([
    model.output[0]
], feed_dict={model.x: _samples, model.dynamics.temperature: 1.})
#_samples

In [ ]:
_samples

In [ ]:
_samples1

In [ ]:
model.train(10)

### Less old

In [ ]:
for key in model.tunneling_rates_highT.keys():
    #print(f'Step num: {key[0]}')
    #print(f'Temp: {key[1]}')
    model.steps_arr.append(key[0])
    model.temp_arr.append(key[1])
    #model.temp_arr.append(key[1])

In [ ]:
for idx in range(len(model.steps_arr)):
    model.steps_arr[idx] += 1

In [ ]:
model.steps_arr

In [ ]:
for key, val in model.tunneling_rates.items():
    model.tunneling_rates_avg.append(val[0])
    model.tunneling_rates_err.append(val[1])
for key, val in model.tunneling_rates_highT.items():
    model.tunneling_rates_avg_highT.append(val[0])
    model.tunneling_rates_err_highT.append(val[1])
    
for key, val in model.acceptance_rates.items():
    model.acceptance_rates_avg.append(val[0])
    model.acceptance_rates_err.append(val[1])
for key, val in model.acceptance_rates_highT.items():
    model.acceptance_rates_avg_highT.append(val[0])
    model.acceptance_rates_err_highT.append(val[1])
    
for key, val in model.distances.items():
    model.distances_avg.append(val[0])
    model.distances_err.append(val[1])
for key, val in model.distances_highT.items():
    model.distances_avg_highT.append(val[0])
    model.distances_err_highT.append(val[1])

In [ ]:
model._save_variables()

In [ ]:
model._init_params(params)

In [ ]:
model._load_variables()

In [ ]:
model.steps_arr

In [ ]:
model.tunneling_rates

In [ ]:
import os

In [ ]:
model.__dict__.keys()

In [ ]:
get_vals_as_arr = lambda _dict: np.array(list(_dict.values()))
tr = get_vals_as_arr(model.tunneling_rates)
ar = get_vals_as_arr(model.acceptance_rates)
dr = get_vals_as_arr(model.distances)
#np.array(list(model.tunneling_rates.values()))

In [ ]:
for key in model.attrs_dict.keys():
    in_file = model.info_dir + key + '.npy'
    if os.path.isfile(in_file):
        setattr(model, key, np.load(in_file))
        print(f'Set model.{key} to values read in from: {in_file}')
        #print(model.key == np.load(in_file))

In [ ]:
model.sess.run(tf.global_variables_initializer())

In [ ]:
attrs_dict = model.attrs_dict

In [ ]:
attrs_dict['steps_arr']

In [ ]:
model.train(params['num_training_steps'], plot=True)

## OLD

In [ ]:
#trajectories, loss_arr, px_arr = model.generate_trajectories(num_samples=100, num_steps=100, temperature=1.)

#config = tf.ConfigProto(log_device_placement=True)
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#model.build_graph()

In [ ]:
def calc_avg_distance1(trajectories):
    distances_arr = []
    for trajectory in trajectories:
        distance_arr = []
        for idx in range(1, len(trajectory)):
            diff = trajectory[idx] - trajectory[idx-1]
            dist = np.sqrt(np.dot(diff, diff.T))
            distance_arr.append(dist)
        distances_arr.append(sum(distance_arr))
    return np.mean(distances_arr)

def calc_avg_distance2(trajectories):
    dist = lambda d: np.sqrt(np.dot(d, d.T))
    #distances_arr = np.mean([[dist(d) for d in [t[:-1, :] - t[1:, :] for t in trajectories]]
    #for trajectory in trajectories:
        diff = trajectory[:-1, :] - trajectory[1:, :]
        distance = sum([np.sqrt(np.dot(d, d.T)) for d in diff])
        distances_arr.append(distance)
    return np.mean(distances_arr)

In [ ]:
%timeit avg_dist = avg_distance_traveled(trajectories.transpose([1,0,2]))
#print(avg_dist)

%timeit avg_dist1 = calc_avg_distance1(trajectories.transpose([1,0,2]))
#print(avg_dist1)

%timeit avg_dist2 = calc_avg_distance2(trajectories.transpose([1,0,2]))
#print(avg_dist2)

distances = np.array([np.sqrt(np.dot(disp, disp.T)) for disp in displacements])

In [ ]:
model.train(params['num_training_steps'], config=config, plot=True)

config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
model.build_graph()
model.train(params['num_training_steps'], config=config, plot=False)